In [2]:
import ifcopenshell
import csv

# Load the IFC file
ifc_file = ifcopenshell.open("Hus28_test.ifc")

# Search for the target storey by name
storey_name = "Plan 10"
target_storey = None

for storey in ifc_file.by_type("IfcBuildingStorey"):
    if storey.Name == storey_name:
        target_storey = storey
        break

if not target_storey:
    print(f"Storey with name '{storey_name}' was not found.")
else:
    # Find all IfcDoor elements
    ifc_doors = ifc_file.by_type("IfcDoor")

    # Initialize a dictionary to store doors and their connected rooms
    door_to_room = {}

    # Iterate over all IfcRelSpaceBoundary elements
    for rel in ifc_file.by_type("IfcRelSpaceBoundary"):
        # Check if the related element is a door
        if rel.RelatedBuildingElement in ifc_doors:
            door = rel.RelatedBuildingElement
            door_oid = door.id()
            
            # Check if the door is on the target storey
            door_storey = door.ContainedInStructure
            if door_storey and door_storey[0].RelatingStructure == target_storey:
                # Assign the room (IfcSpace) to the door
                room = rel.RelatingSpace
                if room:
                    room_name = room.Name
                    if door_oid in door_to_room:
                        door_to_room[door_oid].append(room_name)
                    else:
                        door_to_room[door_oid] = [room_name]

    # Write the results to a CSV file
    with open('Output02_RoomsByDoors.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=';')
        for door_oid, room_names in door_to_room.items():
            csvwriter.writerow([door_oid] + room_names)

print("Results have been saved to Output02_RoomsByDoors.csv")

Results have been saved to Output02_RoomsByDoors.csv
